In [1]:
#获取训练语料
import os

neg_dir=r'comments/10000/neg'
pos_dir=r'comments/10000/pos'

texts=[]
labels=[]

for fname in os.listdir(pos_dir):
    if(fname[-4:]=='.txt'):
        f=open(os.path.join(pos_dir,fname),encoding='utf-8')
        t=f.read()
        texts.append(t.strip().replace('\n',''))
        f.close()
        labels.append(1)
for fname in os.listdir(neg_dir):
    if(fname[-4:]=='.txt'):
        f=open(os.path.join(neg_dir,fname),encoding='utf-8')
        texts.append(f.read())
        f.close()
        labels.append(0)
print(len(texts))
print(texts[100])

10000
入住的是度假区的豪华海景房,前台给了5楼(最高6楼),然后差不多100%的海景,虽然是挂牌5星的,但是本人觉得是4星的标准,和我后来入住的5星喜来登差了蛮多的,不过整体来说还是符合他家的价钱的.


In [2]:
#读取停用词
with open('stopwords.txt', encoding='utf-8') as file:
    words = file.readlines()
stop_words = [word.strip().replace('\n', '') for word in words]
print(len(stop_words))
print(stop_words[500])

2612
八


In [4]:
#文本预处理
import jieba
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

MAX_WORDS=100000 #使用100000个词语
MAX_LEN=100 #每条评论的长度


#将句子分词并用空格隔开
textList=[' '.join([w for w in jieba.cut(text) if w not in stop_words]) for text in texts]

tokenizer=Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(texts=textList)
sequences=tokenizer.texts_to_sequences(texts=textList)

word_index=tokenizer.word_index

data=pad_sequences(sequences=sequences,maxlen=MAX_LEN)
labels=np.asarray(labels)
print(data.shape)
print(data[10])

(10000, 100)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     5   179   185   179  1092   199     8  7055  1763
  1391 17560  8784     8   202 17561  6125  7056  8785 17562   722 17563
   899 17564 17565     8  1169    68    88   145  1710   739  3214  8786
  2521  3412  1198   423  6126   194  1313  2409  1112    36    39  1429
  5352  3998  1616   657  4825  8787   657   423  3649  7057   173  7058
   683   657  1170    89   171  4358   488  5353 17566  7059   301   488
   423  3649 10963 17567]


In [5]:
#划分训练集、验证集、测试集

#各数据集数量
TRAIN_SAMPLES=6000
VALIDATION_SAMPLES=2000
TEST_SAMPLES=2000

#打乱数据集
indices=np.arange(data.shape[0])
np.random.shuffle(indices)
data=data[indices]
labels=labels[indices]

x_train=data[:TRAIN_SAMPLES]
y_train=labels[:TRAIN_SAMPLES]

x_val=data[TRAIN_SAMPLES:TRAIN_SAMPLES+VALIDATION_SAMPLES]
y_val=labels[TRAIN_SAMPLES:TRAIN_SAMPLES+VALIDATION_SAMPLES]

x_test=data[TRAIN_SAMPLES+VALIDATION_SAMPLES:]
y_test=labels[TRAIN_SAMPLES+VALIDATION_SAMPLES:]

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(6000, 100)
(2000, 100)
(2000, 100)


In [6]:
#解析词向量文件


#映射字典，每个单词Key对应一个词向量
embeddings_index={}

f=open('sgns.zhihu.word',encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]#第一个是单词
    coefs=np.asarray(values[1:],dtype='float32')#后面都是系数
    embeddings_index[word]=coefs
f.close()
print(len(embeddings_index))
print(len(embeddings_index['的']))

259837
300


In [7]:
#准备词向量矩阵

EMBEDDING_DIM=300#词向量的长度

embedding_matrix=np.zeros((MAX_WORDS,EMBEDDING_DIM))

for word,i in word_index.items():
    word_vector=embeddings_index.get(word)
    if(word_vector is not None):#若是未登录词，则词向量为初始值0
        embedding_matrix[i]=word_vector
print(embedding_matrix.shape)

(100000, 300)


In [20]:
#定义模型
from keras.models import Sequential
from keras.layers import *

model=Sequential()
model.add(Embedding(MAX_WORDS,EMBEDDING_DIM,input_length=MAX_LEN))#词嵌入层
#model.add(Conv1D(128,3,activation='relu'))
#model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 300)          30000000  
_________________________________________________________________
flatten_4 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               7680256   
_________________________________________________________________
dense_17 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_18 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_19 (Dense)             (None, 32)                8224      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 33        
Total para

In [21]:
#把词嵌入矩阵载入到词嵌入层中

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable=False#冻结


In [23]:
#编译并训练模型
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history=model.fit(x_train,y_train,epochs=50,batch_size=1024,validation_data=(x_val,y_val))
model.save_weights('hotel_comments_classify.h5')

Train on 6000 samples, validate on 2000 samples
Epoch 1/50
6000/6000 [==============================] - 1s 241us/step - loss: 2.0213 - acc: 0.8000 - val_loss: 0.4633 - val_acc: 0.8520
Epoch 2/50
6000/6000 [==============================] - 0s 60us/step - loss: 0.0162 - acc: 0.9977 - val_loss: 0.4887 - val_acc: 0.8585
Epoch 3/50
6000/6000 [==============================] - 0s 60us/step - loss: 0.0084 - acc: 0.9997 - val_loss: 0.5342 - val_acc: 0.8585
Epoch 4/50
6000/6000 [==============================] - 0s 61us/step - loss: 0.0047 - acc: 0.9998 - val_loss: 0.6200 - val_acc: 0.8570
Epoch 5/50
6000/6000 [==============================] - 0s 61us/step - loss: 0.0024 - acc: 0.9998 - val_loss: 0.7295 - val_acc: 0.8560
Epoch 6/50
6000/6000 [==============================] - 0s 61us/step - loss: 0.0013 - acc: 0.9998 - val_loss: 0.8004 - val_acc: 0.8580
Epoch 7/50
6000/6000 [==============================] - 0s 60us/step - loss: 8.2684e-04 - acc: 0.9998 - val_loss: 0.8675 - val_acc: 0.8575
Ep

In [24]:
#测试模型

model.evaluate(x_test,y_test)

2000/2000 [==============================] - 0s 237us/step


[1.2969743014619453, 0.8685]